### **[LeetCode Link](https://leetcode-cn.com/problems/regular-expression-matching/solution/zheng-ze-biao-da-shi-pi-pei-by-leetcode-solution/)**

## 动态规划
### 思路与算法
题目中的匹配是一个「逐步匹配」的过程：我们每次从字符串 $p$ 中取出一个字符或者「字符 + 星号」的组合，并在 $s$ 中进行匹配。对于 $p$ 中一个字符而言，它只能在 $s$ 中匹配一个字符，匹配的方法具有唯一性；而对于 $p$ 中字符 + 星号的组合而言，它可以在 $s$ 中匹配任意自然数个字符，并不具有唯一性。因此我们可以考虑使用动态规划，对匹配的方案进行枚举。

我们用 $f[i][j]$ 表示 $s$ 的前 $i$ 个字符与 $p$ 中的前 $j$ 个字符是否能够匹配。在进行状态转移时，我们考虑 $p$ 的第 $j$ 个字符的匹配情况：
* 如果 $p$ 的第 $j$ 个字符是一个小写字母，那么我们必须在 $s$ 中匹配一个相同的小写字母，即
<center>$f[i][j] = \begin{cases} f[i - 1][j - 1], & s[i] = p[j]\\ \text{false}, & s[i] \neq p[j] \end{cases}$</center>
 也就是说，如果 $s$ 的第 $i$ 个字符与 $p$ 的第 $j$ 个字符不相同，那么无法进行匹配；否则我们可以匹配两个字符串的最后一个字符，完整的匹配结果取决于两个字符串前面的部分。
* 如果 $p$ 的第 $j$ 个字符是 `*`，那么就表示我们可以对 $p$ 的第 $j-1$ 个字符匹配任意自然数次。在匹配 $0$ 次的情况下，我们有
<center>$f[i][j] = f[i][j - 2]$</center>
 也就是我们「浪费」了一个字符 + 星号的组合，没有匹配任何 $s$ 中的字符。
 
 在匹配 $1,2,3, \cdots$ 次的情况下，类似地我们有
<center>$\begin{aligned} & f[i][j] = f[i - 1][j - 2], \quad && \text{if~} s[i] = p[j - 1] \\ & f[i][j] = f[i - 2][j - 2], \quad && \text{if~} s[i - 1] = s[i] = p[j - 1] \\ & f[i][j] = f[i - 3][j - 2], \quad && \text{if~} s[i - 2] = s[i - 1] = s[i] = p[j - 1] \\ & \cdots\cdots & \end{aligned}$</center>
 如果我们通过这种方法进行转移，那么我们就需要枚举这个组合到底匹配了 $s$ 中的几个字符，会导致时间复杂度增加，并且代码编写起来十分麻烦。我们不妨换个角度考虑这个问题：字母 + 星号的组合在匹配的过程中，本质上只会有两种情况：
 * 匹配 $s$ 末尾的一个字符，将该字符扔掉，而该组合还可以继续进行匹配；
 * 不匹配字符，将该组合扔掉，不再进行匹配。

 如果按照这个角度进行思考，我们可以写出很精巧的状态转移方程：
<center>$f[i][j] = \begin{cases} f[i - 1][j] \text{ or } f[i][j - 2], & s[i] = p[j - 1] \\ f[i][j - 2], & s[i] \neq p[j - 1]\end{cases}$</center>
 在任意情况下，只要 $p[j]$ 是 `.`，那么 p[j]p[j] 一定成功匹配 $s$ 中的任意一个小写字母。

最终的状态转移方程如下：
<center>$f[i][j] = \begin{cases} \text{if} (p[j] \neq \text{`*'}) = \begin{cases} f[i - 1][j - 1], & \textit{matches}(s[i], p[j])\\ \text{false}, & \text{otherwise} \end{cases} \\ \text{otherwise} = \begin{cases} f[i - 1][j] \text{ or } f[i][j - 2], & \textit{matches}(s[i], p[j-1]) \\ f[i][j - 2], & \text{otherwise} \end{cases} \end{cases}$</center>
其中 $\textit{matches}(x, y)$ 判断两个字符是否匹配的辅助函数。只有当 $y$ 是 `.` 或者 $x$ 和 $y$ 本身相同时，这两个字符才会匹配。

### 复杂度分析
* 时间复杂度：$O(mn)$，其中 $m$ 和 $n$ 分别是字符串 $s$ 和 $p$ 的长度。我们需要计算出所有的状态，并且每个状态在进行转移时的时间复杂度为 $O(1)$。
* 空间复杂度：$O(mn)$，即为存储所有状态使用的空间。

In [ ]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        m, n = len(s), len(p)

        def matches(i: int, j: int) -> bool:
            if i == 0:
                return False
            if p[j - 1] == '.':
                return True
            return s[i - 1] == p[j - 1]

        f = [[False] * (n + 1) for _ in range(m + 1)]
        f[0][0] = True
        for i in range(m + 1):
            for j in range(1, n + 1):
                if p[j - 1] == '*':
                    f[i][j] |= f[i][j - 2]
                    if matches(i, j - 1):
                        f[i][j] |= f[i - 1][j]
                else:
                    if matches(i, j):
                        f[i][j] |= f[i - 1][j - 1]
        return f[m][n]

In [ ]:
class RegularExpressionMatching {
    public boolean isMatch(String s, String p) {
        int m = s.length();
        int n = p.length();

        boolean[][] f = new boolean[m + 1][n + 1];
        f[0][0] = true;
        for (int i = 0; i <= m; ++i) {
            for (int j = 1; j <= n; ++j) {
                if (p.charAt(j - 1) == '*') {
                    f[i][j] = f[i][j - 2];
                    if (matches(s, p, i, j - 1)) {
                        f[i][j] = f[i][j] || f[i - 1][j];
                    }
                } else {
                    if (matches(s, p, i, j)) {
                        f[i][j] = f[i - 1][j - 1];
                    }
                }
            }
        }
        return f[m][n];
    }

    public boolean matches(String s, String p, int i, int j) {
        if (i == 0) {
            return false;
        }
        if (p.charAt(j - 1) == '.') {
            return true;
        }
        return s.charAt(i - 1) == p.charAt(j - 1);
    }
}

### **[LeetCode Link](https://leetcode-cn.com/problems/regular-expression-matching/solution/jian-ming-qing-xi-xie-fa-python3xiang-xi-zhu-shi-b/452865)**

效率很不好看，优化一下：
* 记忆化搜索，避免重复递归运算（动态规划）；
* 用索引号代替切片，减少cache过程本身的时间成本；
* 最大限度维持原先代码的简洁逻辑。

### 时间复杂度：
* 改进前：平均$O(n \times 2^\text{星号数})$，最差$O(2^n)$，
* 改进后：平均$O(n \times (\text{星号数+1}))$，最差$O(n)$

In [ ]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        # 装饰符实现记忆化搜索，等价于Top-Down动态规划
        @lru_cache(None)
        def recur(i,j):
            # 结束条件
            if j==len(p): return i==len(s)
            # 首字母匹配
            first_match = (len(s) > i) and (p[j] == s[i] or p[j] == '.')
            # 处理 `*`
            if len(p) >=j+2 and p[j+1] == '*':
                return recur(i, j+2) or (first_match and recur(i+1, j))
            # 处理首字母匹配
            return first_match and recur(i+1, j+1)
        return recur(0,0)